In [1]:
start_date='2020-07-11'
end_date='2023-07-11'
geometry=(92.76,26.54)

In [ ]:
import ee
import geemap
import pandas as pd

class paired_labels():
    def __init__(self, start_date, end_date, geometry):
        self.start_date=ee.Date(start_date)
        self.end_date=ee.Date(end_date)
        self.geometry=ee.Geometry.Point(geometry)
    def Initialize(self):
    # Initialize the Earth Engine module.
        try:
            ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
        except Exception as e:
            ee.Authenticate()
            ee.Initialize()
        print('Earth Engine initialized.')
    def get_dates_s2(self,image):
        date=image.date().format()
        return ee.Feature(None, {'date':date})
    def get_dates_s1(self,image):
        date=ee.Date(image.get('system:time_start')).format().slice(0,10)
        return ee.Feature(None, {'date':date})
    
    def create_pairs_dataframe(self):
        s2=ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filter((ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))).filterDate(start_date,end_date).filterBounds(geometry)
        s1=ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(start_date,end_date).filterBounds(geometry)
        dates_sen2=s2.map(self.get_dates_s2).getInfo()
        dates_sen1=s1.map(self.get_dates_s1).getInfo()
        return dates_sen2,dates_sen1
Initialize()

In [ ]:
p=paired_labels(start_date,end_date,geometry)
p.create_pairs_dataframe()

In [ ]:

ee_geometry=ee.Geometry.Point(geometry)
s1=ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(ee.Date('2017-01-01'),ee.Date('2024-04-01')).filterBounds(ee_geometry)
def get_dates_s1(image):
    date=ee.Date(image.get('system:time_start')).format().slice(0,10)
    return ee.Feature(None, {'date':date})
fc_sen1=s1.map(get_dates_s1).getInfo()


In [ ]:
# ee_date=ee.Date(date)

s2=ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filter((ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))).filterDate(ee.Date('2017-01-01'),ee.Date('2024-04-01')).filterBounds(ee_geometry)

In [ ]:
df=pd.DataFrame(columns=['Date'])
for i in range(len(fc['features'])):
    df.loc[len(df)]=fc['features'][i]['properties']['date']
df['Date']=pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

In [ ]:
df1=pd.DataFrame(columns=['Date'])
for i in range(len(fc_sen1['features'])):
    df1.loc[len(df1)]=fc_sen1['features'][i]['properties']['date']
df1['Date']=pd.to_datetime(df1['Date']).dt.strftime('%Y-%m-%d')

In [ ]:

s1=ee.ImageCollection("COPERNICUS/S1_GRD").filterDate(ee.Date('2017-01-01'),ee.Date('2024-04-01')).filterBounds(ee_geometry)
def get_dates_s1(image):
    date=ee.Date(image.get('system:time_start')).format().slice(0,10)
    return ee.Feature(None, {'date':date})
fc_sen1=s1.map(get_dates_s1).getInfo()

In [ ]:
df3 = pd.merge(df1, df, how='inner', left_on='Date', right_on='Date')
df3.reset_index(inplace=True, drop=True)